In [29]:
import pandas as pd

# Regressão Linear
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#Arvore de decisão
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('Usuario fez upload do dataset "{name}" com tamanho {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving giroscope_mpu_off.csv to giroscope_mpu_off.csv
Saving giroscope_mpu_on.csv to giroscope_mpu_on.csv
Usuario fez upload da imagem "giroscope_mpu_off.csv" com tamanho 31645 bytes
Usuario fez upload da imagem "giroscope_mpu_on.csv" com tamanho 2974 bytes


In [7]:
df_off = pd.read_csv('/content/giroscope_mpu_off.csv')
df_off=df_off.assign(Outcome=1) #Outcome = 1 é para os dados da tabela OFF
df_off.head()

,id,x,y,z,date_mpu,time_mpu,time_float,Outcome
0,101,-1412,1064,15756,2022-03-25,20:20:59,20.3497,1
1,102,-1232,1000,15732,2022-03-25,20:21:16,20.3544,1
2,103,-1112,1000,15648,2022-03-25,20:21:32,20.3589,1
3,104,-1164,1000,15684,2022-03-25,20:21:48,20.3633,1
4,105,-1196,892,15768,2022-03-25,20:22:03,20.3675,1


In [6]:
df_on = pd.read_csv('/content/giroscope_mpu_on.csv')
df_on=df_on.assign(Outcome=0) #Outcome = 0 é para os dados da tabela ON
df_on.head()

,id,x,y,z,date_mpu,time_mpu,time_float,Outcome
0,1,-852,-6288,13332,2022-03-24,17:08:00,17.1333,0
1,2,-11240,-4752,11148,2022-03-24,17:08:18,17.1383,0
2,3,288,-1656,8444,2022-03-24,17:08:33,17.1425,0
3,4,6144,-4272,17420,2022-03-24,17:08:49,17.1469,0
4,5,1420,900,15400,2022-03-24,17:09:05,17.1514,0


In [9]:
df = pd.concat([df_off, df_on])
df.head()

,id,x,y,z,date_mpu,time_mpu,time_float,Outcome
0,101,-1412,1064,15756,2022-03-25,20:20:59,20.3497,1
1,102,-1232,1000,15732,2022-03-25,20:21:16,20.3544,1
2,103,-1112,1000,15648,2022-03-25,20:21:32,20.3589,1
3,104,-1164,1000,15684,2022-03-25,20:21:48,20.3633,1
4,105,-1196,892,15768,2022-03-25,20:22:03,20.3675,1


In [10]:
# Quantos dados? 
df.shape

(702, 8)

In [11]:
# Quais os tipos? Dados faltantes? 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 702 entries, 0 to 59
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          702 non-null    int64  
 1   x           702 non-null    int64  
 2   y           702 non-null    int64  
 3   z           702 non-null    int64  
 4   date_mpu    702 non-null    object 
 5   time_mpu    702 non-null    object 
 6   time_float  702 non-null    float64
 7   Outcome     702 non-null    int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 49.4+ KB


In [20]:
# Apenas colunas necessarias

df = df[{'id',	'x',	'y',	'z', 'Outcome'}]
df.head()

,z,y,id,Outcome,x
0,15756,1064,101,1,-1412
1,15732,1000,102,1,-1232
2,15648,1000,103,1,-1112
3,15684,1000,104,1,-1164
4,15768,892,105,1,-1196


# **Modelagem - Regressão Linear**

In [21]:
# Separação de DADOS e LABEL
X = df.drop(columns = ["Outcome"])
Y = df["Outcome"]

In [22]:
# Dividindo dados para TREINO e TESTE
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, 
                                                    random_state=42)

In [23]:
# Treinando o modelo
modelo = LinearRegression()
modelo.fit(X_train, Y_train)

LinearRegression()

In [24]:
# Fazendo as predições
Y_pred = modelo.predict(X_test)

# **Avaliação**

In [25]:
# Erro quadrático médio
mean_squared_error(Y_test, Y_pred)

0.033460417678235405

In [26]:
# Erro absoluto médio
mean_absolute_error(Y_test, Y_pred)

0.12000754743351631

In [27]:
# R²
r2_score(Y_test, Y_pred)

0.5225348540215645

# Arvore de **Decissão**

In [30]:
X = df.drop(columns="Outcome") # Aqui temos os DADOS 
y = df["Outcome"]              # Aqui temos os LABELS

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [32]:
# Arvore 
tree = DecisionTreeClassifier()     # Definiu o construtor do método
tree.fit(X_train, y_train)          # Treinou o método nos dados do treino
Y_pred_label = tree.predict(X_test) # Usou o modelo treinado para fazer uma predição dos dados de teste

In [33]:
# Quais são as classes? 
tree.classes_

array([0, 1])

In [35]:
# Quais são os labels preditos? 
Y_pred_label

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
# Quais as métricas? 
print(metrics.classification_report(y_test, Y_pred_label))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00       163

    accuracy                           1.00       176
   macro avg       1.00      1.00      1.00       176
weighted avg       1.00      1.00      1.00       176



In [37]:
# Modelo de florestas aleatórias

floresta = RandomForestClassifier(n_estimators=5) # Cria a floresta, com "n_estimators" árvores
floresta.fit(X_train, y_train)                    # Treina o modelo
Y_pred_label_floresta = floresta.predict(X_test)  # Faz uma predição dos dados de teste

In [38]:
print(metrics.classification_report(y_test, Y_pred_label_floresta))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00       163

    accuracy                           1.00       176
   macro avg       1.00      1.00      1.00       176
weighted avg       1.00      1.00      1.00       176

